In [12]:
from bs4 import BeautifulSoup
import requests

In [45]:
url = 'https://en.bab.la/dictionary/indonesian-english/'

In [77]:
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language':'en-US,en;q=0.9,vi;q=0.8',
    # 'Cache-Control':'max-age=0',
    # 'Cookie': 'PHPSESSID=bocbqfnnlq55gfufvfv0rs4kg8; __cflb=02DiuEFB1Cip5Z9bPzyYpFmAjuGFgx9fxMd7oF26siqBS; _sp_ses.4bba=*; cf_clearance=T0RemuOT.HedXY7Ey.Zoj9QJmOtBstWlvw.oFNWZ3wU-1723442723-1.0.1.1-__p4hgFV.KCOORpUtCtzvMkg2F_8tHmpZ7GtH2dcIgd6o1Jzv9vMNApyA81IpFwI5HXmAA3MqHhCCdIhlD.UCA; languages=en.iden.vi.en.0...iden%7C; __cf_bm=x9MZvqDMQF0g3jM9YmYpS5Bs_mqvCMN.Iw3HYdv1kOc-1723446326-1.0.1.1-YMROabdHNfA6Lz8.w6uISy_Qj31YsJP02oa4LbLDUtvaQj98KBoZrdo162pLSSFg1xBCc0G5vazvvN93mXkdoQ; OptanonConsent=isGpcEnabled=0&datestamp=Mon+Aug+12+2024+14%3A05%3A27+GMT%2B0700+(Indochina+Time)&version=202407.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&geolocation=VN%3BHN&AwaitingReconsent=false; OptanonAlertBoxClosed=2024-08-12T07:05:27.529Z; AWSALB=61g0mSrYlsxWt3W3GhPDTH/iFfsjipUrh6s3iJnULna69YVbYpFaACCprCVmWlGw8Q/YQbUYbHDlHzEbszrhcmMUoZzeKwlzFHNXZNrsPPnUC1akIQKLy/byGtff; AWSALBCORS=61g0mSrYlsxWt3W3GhPDTH/iFfsjipUrh6s3iJnULna69YVbYpFaACCprCVmWlGw8Q/YQbUYbHDlHzEbszrhcmMUoZzeKwlzFHNXZNrsPPnUC1akIQKLy/byGtff; _sp_id.4bba=14469fb6-a6d5-47b7-8d42-5da1b97ac788.1723436617.1.1723447197..98fd95be-d67e-4d0c-9cd5-34930faf61a3..77df6a0e-1998-4997-968c-65f6bd755ee8.1723436616953.205',
    # 'Priority':'u=0, i',
    # 'Referer':'https://en.bab.la/dictionary/indonesian-english/c/1'
    # 'Sec-Ch-Ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Microsoft Edge";v="126"',
    # 'Sec-Ch-Ua-Mobile':'?0',
    # 'Sec-Ch-Ua-Platform': "Linux",
    'Sec-Fetch-Dest':'document',
    'Sec-Fetch-Mode':'navigate',
    'Sec-Fetch-Site':'same-origin',
    'Sec-Fetch-User':'?1',
    'Upgrade-Insecure-Requests':'1',
    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0'
}

In [92]:
page = requests.get(url,headers=headers )
print(page.status_code)
print(page.request.headers)

200
{'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7', 'Connection': 'keep-alive', 'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8', 'Sec-Fetch-Dest': 'document', 'Sec-Fetch-Mode': 'navigate', 'Sec-Fetch-Site': 'same-origin', 'Sec-Fetch-User': '?1', 'Upgrade-Insecure-Requests': '1'}


In [80]:
def get_next_button(html:BeautifulSoup) -> str | None:
    """
    Return the next button, if there are not return None
    """

    next = html.find_all('a', class_ = 'dict-pag-button')
    for a in next:
        if a.text == '>':
            return a['href']
    return None

In [87]:
def get_access(url, max_iters, header):
    status_code = 404
    iters = 0
    while status_code != 200:
        page = requests.get(url,headers=header)
        status_code = page.status_code
        iters+=1
        if iters > max_iters:
            print(f"Failed to access url after {max_iters} attemps\nHTTP Response: {status_code}")
            return None
    print(f"Succesfully access url with {page.status_code}")
    return page

In [91]:
def write_word(page_html, file_name):
    columns = page_html.find_all('div', class_ = 'dict-select-column')
    for column in columns:
        for li in column.find_all('li'):
            with open(file_name,'a') as text_file:
                text_file.write(f'{li.text[11:]}\n')

In [88]:
def get_words(url:str, header:dict ,alphabet:list=0,max_iters=20,folder:str="") -> None:
    """
    Try to get all word and save to a folder
    """
    if alphabet==0:
        alphabet = 'A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z'
        alphabet = alphabet.lower().split(',')

    for letter in alphabet:
        page_num = 1
        current_url = f'{url}/{letter}/{page_num}'
    
        while current_url != None:
            page = get_access(current_url,max_iters,header)
            if page == None:
                return None
            page_html = BeautifulSoup(page.text, 'html')
            write_word(page_html,f'{letter}.txt')
            print(f'printed {letter} for the {page_num}th')
            page_num += 1
            
            next_page = get_next_button(page_html)
            current_url = f'https:{next_page}' if next_page != None else None 

In [93]:
get_words(url, headers)

Succesfully access url with 200
printed a for the 1th
Succesfully access url with 200
printed a for the 2th
Succesfully access url with 200
printed a for the 3th
Succesfully access url with 200
printed a for the 4th
Succesfully access url with 200
printed a for the 5th
Succesfully access url with 200
printed a for the 6th
Succesfully access url with 200
printed a for the 7th
Succesfully access url with 200
printed a for the 8th
Succesfully access url with 200
printed b for the 1th
Succesfully access url with 200
printed b for the 2th
Succesfully access url with 200
printed b for the 3th
Succesfully access url with 200
printed b for the 4th
Succesfully access url with 200
printed b for the 5th
Succesfully access url with 200
printed b for the 6th
Succesfully access url with 200
printed b for the 7th
Succesfully access url with 200
printed b for the 8th
Succesfully access url with 200
printed b for the 9th
Succesfully access url with 200
printed b for the 10th
Succesfully access url with